# 包括的なコンテンツと長文出力の生成

このノートブックでは、Claude 3.7 Sonnet の拡張思考機能を使用して、高品質の長文コンテンツを生成する方法を説明します。学習内容:

1. 包括的なビジネス レポートを生成する方法
2. クリエイティブ ライティングとストーリーテリングのテクニック
3. 技術ドキュメントのアプローチ
4. 長い出力を管理および検証するためのベスト プラクティス

## 前提条件

- Claude 3.7 の拡張思考機能に関する基本的な理解 (以前のレッスンで説明)
- Bedrock API とユーティリティ関数に関する知識
- Python プログラミングの経験

まず、環境を設定して必要なライブラリをインポートしましょう。

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## 長文コンテンツ生成の理解

特定のコンテンツ タイプについて詳しく説明する前に、Claude 3.7 の拡張思考機能が長文コンテンツ生成をどのように強化するかを理解することが重要です。

1. **構造化された計画**: Claude は、コンテンツを生成する前に詳細なアウトラインを作成できます
2. **一貫性のあるコンテキスト**: 拡張思考により、長いコンテンツ間で一貫性を維持できます
3. **反復的な開発**: 段階的にコンテンツを開発し、段階的に改良することができます
4. **コンテンツの検証**: 組み込みのメカニズムにより、品質と完全性が確保されます

構造化されたビジネス レポートを生成する簡単な例から始めましょう。

### ユーティリティ関数に関する注意

このレッスンでは、コンテンツの分析と検証に、`claude_utils.py` モジュールのいくつかのヘルパー関数を使用します。

- `analyze_technical_doc`: 技術文書の構造とコンポーネントを分析します
- `analyze_creative_writing`: クリエイティブ ライティングの構造と要素を分析します
- `validate_long_form_content`: コンテンツの構造と完全性の基本的な検証を実行します

これらのユーティリティは、コンテンツ分析の「舞台裏」の作業を処理し、Claude 3.7 の拡張思考機能を使用してさまざまな種類の長文コンテンツを生成するというコア コンセプトに集中できるようにします。

これらのユーティリティ関数の実装の詳細に関心がある場合は、`claude_utils.py` ファイルで見つけることができます。ただし、このレッスンの概念を学習するために、それらの実装を理解する必要はありません。

では、包括的なコンテンツを生成する方法を調べてみましょう...

## 1. ビジネス レポートの生成

まずは包括的なビジネス レポートの作成から始めましょう。分析、データの表示、推奨事項を必要とする実際のシナリオを使用します。

まず、レポート生成のベスト プラクティスを実装するヘルパー関数を作成します。

1. 明確なアウトラインから始める
2. 構造化されたセクションを使用する
3. データ分析を含める
4. 実用的な推奨事項を提供する

In [ ]:
def generate_report(prompt, reasoning_budget=4096, max_tokens=2000):
    """
    Generate a comprehensive business report using Claude's extended thinking
    
    Args:
        prompt (str): The report topic and requirements
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        str: The generated report
    """
    # Create a structured prompt that encourages good report format
    structured_prompt = f"""
    Please create a comprehensive business report on the following topic:
    
    {prompt}
    
    Follow this structure:
    1. Executive Summary
    2. Background and Context
    3. Analysis and Findings
    4. Recommendations
    5. Implementation Plan
    6. Conclusion
    
    Make each section detailed and thorough. Support your analysis with clear reasoning.
    Format the report using markdown for better readability.
    """
    
    # Invoke Claude with extended thinking
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        structured_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### レポート生成のテスト

リモートワークが従業員の生産性と企業文化に与える影響を分析するという現実的なビジネス シナリオでレポート生成機能をテストしてみましょう。

この例では、Claude が次のことができることを説明します。
- 構造化された分析を開発する
- 複数の要因を考慮する
- データに基づく洞察を提供する
- 実用的な推奨事項を作成する

In [ ]:
# Define our report topic
report_topic = """
Analyze the impact of remote work on employee productivity and company culture at a mid-sized technology company (500 employees).
Consider:
- Productivity metrics before and after remote work adoption
- Employee satisfaction and retention
- Communication and collaboration effectiveness
- Impact on hiring and recruitment
- Cost implications (office space, utilities, equipment)

Provide specific recommendations for optimizing the hybrid work model going forward.
"""

# Generate the report
report_content, report_response = generate_report(
    report_topic,
    reasoning_budget=6144,  # Larger budget for detailed analysis
    max_tokens=2500        # Allow for longer output
)

# Display the report with metrics
print("Report Generation Metrics:")
print(f"Time taken: {report_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Input tokens: {report_response.get('usage', {}).get('inputTokens', 0)}")
print(f"Output tokens: {report_response.get('usage', {}).get('outputTokens', 0)}\n")

# Display the report content
display(Markdown(report_content))

### レポート品質の分析

最初のレポートを生成したので、その効果的な理由と、さらに改善する方法を検討してみましょう。考慮すべき主な側面:

1. **構造**: コンテンツはどの程度整理されていますか?
2. **深さ**: トピックは徹底的に調査されていますか?
3. **裏付けとなる証拠**: 主張はどの程度裏付けられていますか?
4. **実行可能性**: 推奨事項はどの程度実用的ですか?

レポート品質の評価と改善に役立つ関数を作成しましょう。

In [ ]:
def analyze_report_quality(report_content):
    """
    Analyze the quality of a generated report
    
    Args:
        report_content (str): The report content to analyze
        
    Returns:
        dict: Quality metrics and suggestions for improvement
    """
    # Split the report into sections
    sections = report_content.split('\n#')
    
    # Basic metrics
    metrics = {
        "section_count": len(sections),
        "total_length": len(report_content),
        "sections": {}
    }
    
    # Analyze each section
    for section in sections:
        if section.strip():
            # Get section title (first line)
            title = section.split('\n')[0].strip('# ')
            content = '\n'.join(section.split('\n')[1:])
            
            metrics["sections"][title] = {
                "length": len(content),
                "paragraphs": len([p for p in content.split('\n\n') if p.strip()]),
                "bullet_points": len([l for l in content.split('\n') if l.strip().startswith('-')])
            }
    
    return metrics

def display_report_analysis(metrics):
    """Display report analysis in a readable format"""
    display(Markdown("""
    ### Report Analysis
    
    #### Overall Metrics
    - Total Length: {} characters
    - Number of Sections: {}
    
    #### Section Breakdown
    """.format(metrics["total_length"], metrics["section_count"])))
    
    for section, data in metrics["sections"].items():
        display(Markdown(f"""
        **{section}**
        - Length: {data['length']} characters
        - Paragraphs: {data['paragraphs']}
        - Bullet Points: {data['bullet_points']}
        """))

In [ ]:
# Analyze our generated report
report_metrics = analyze_report_quality(report_content)
display_report_analysis(report_metrics)

# Let's also create an improved prompt based on this analysis
def create_enhanced_report_prompt(topic, metrics):
    """
    Create an improved prompt based on report analysis
    """
    # Calculate average section length for balance
    avg_section_length = metrics["total_length"] / max(metrics["section_count"], 1)
    
    enhanced_prompt = f"""
    Please create a comprehensive business report on:
    
    {topic}
    
    Requirements for a well-balanced report:
    1. Each major section should be approximately {int(avg_section_length/1.5)}-{int(avg_section_length*1.5)} characters
    2. Include 2-3 paragraphs per section for depth
    3. Use bullet points for key findings and recommendations
    4. Support claims with specific examples or data
    5. Include clear, actionable next steps
    
    Format using markdown with:
    - Clear section headers (# for main sections, ## for subsections)
    - Bullet points for lists
    - Bold text for key points
    - Tables for data presentation where appropriate
    """
    
    return enhanced_prompt

### レポート生成の改善

分析により、レポート生成を強化できる方法がいくつかわかりました。

1. **セクションのバランスの改善**: セクションの深さを同じにする
2. **裏付けとなる証拠を増やす**: 具体的なデータ ポイントと例を含める
3. **構造の強化**: サブセクションと書式を使用してわかりやすくする
4. **実用的な詳細**: 推奨事項をより具体的にする

これらの洞察を使用して、レポートの改善版を生成してみましょう。

In [ ]:
# Create an enhanced prompt using our insights
enhanced_prompt = create_enhanced_report_prompt(report_topic, report_metrics)

# Generate an improved report with a larger reasoning budget
improved_report_content, improved_response = generate_report(
    enhanced_prompt,
    reasoning_budget=8192,  # Increased for more thorough analysis
    max_tokens=3000       # Increased for more detailed content
)

# Analyze the improved report
improved_metrics = analyze_report_quality(improved_report_content)

# Compare the original and improved reports
print("Comparison of Original vs. Improved Report:")
print(f"Original Report Sections: {report_metrics['section_count']}")
print(f"Original Report Length: {report_metrics['total_length']} characters")
print(f"Improved Report Sections: {improved_metrics['section_count']}")
print(f"Improved Report Length: {improved_metrics['total_length']} characters\n")

# Display the improved report
display(Markdown("### Improved Report"))
display(Markdown(improved_report_content))

## チェックイン ポイント: レポート生成

他のコンテンツ タイプに進む前に、効果的なレポート生成についての理解を確認しましょう。

**重要な洞察:**
1. **構造が重要**: バランスの取れたコンテンツでセクションを整理
2. **証拠に基づく**: データと例で主張を裏付ける
3. **段階的な詳細**: 高レベルの要約から具体的な推奨事項に移行する
4. **読みやすい形式**: マークダウン形式を効果的に使用する

**検討すべき質問:**
- 推論予算の増加はレポートの品質にどのように影響しましたか?
- レポートの有用性に最も影響を与えた改善点はどれですか?
- このアプローチからメリットを得られる他の種類のビジネス レポートはどれですか?

では、次のコンテンツ タイプであるクリエイティブ ライティングに移りましょう。

## 2. クリエイティブ ライティングとストーリーテリング

クリエイティブ ライティングには、ビジネス レポートとは異なる課題があります。次の点に重点を置く必要があります。
- キャラクター開発
- プロット構造
- 説明的な言語
- 一貫した語り口

長い文章でも品質と一貫性が維持されるクリエイティブ コンテンツを作成するためのフレームワークを作成しましょう。

In [ ]:
def generate_creative_content(
    prompt,
    content_type="story",  # story, script, or poem
    reasoning_budget=6144,
    max_tokens=2500
):
    """
    Generate creative content using Claude's extended thinking
    
    Args:
        prompt (str): The creative writing prompt
        content_type (str): Type of creative content to generate
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        tuple: (content, response object)
    """
    # Create a structured prompt for creative writing
    creative_prompt = f"""
    Please create a piece of {content_type} based on the following prompt:
    
    {prompt}
    
    Before writing, create a detailed outline including:
    1. Characters/Elements
    2. Plot points/Structure
    3. Key themes
    4. Important descriptions
    
    Then develop the {content_type} with attention to:
    - Consistent narrative voice
    - Vivid descriptive language
    - Natural dialogue (if applicable)
    - Pacing and flow
    
    Use markdown formatting for structure and emphasis.
    """
    
    # Invoke Claude with extended thinking
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        creative_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### クリエイティブ ライティング生成のテスト

短編小説のプロンプトを使用してクリエイティブ ライティング機能をテストしてみましょう。これにより、Claude が次のことができることが示されます。
- キャラクターとプロットを開発する
- 物語の一貫性を維持する
- 魅力的な説明を作成する
- 見せることと伝えることのバランスをとる

いくつかの初期要素を提供しながらも創造性の余地を残したプロンプトから始めます。

In [ ]:
# Define our story prompt
story_prompt = """
Create a short story about a scientist who discovers an unusual pattern in their data 
that suggests something impossible is happening in their lab late at night.

The story should:
- Build suspense gradually
- Include both scientific and personal elements
- Have a surprising but satisfying resolution
- Be between 1000-1500 words

Main character traits:
- Dedicated researcher
- Initially skeptical
- Personally invested in their work
"""

# Generate the story
story_content, story_response = generate_creative_content(
    story_prompt,
    content_type="story",
    reasoning_budget=8192,  # Larger budget for creative development
    max_tokens=3000        # Allow for longer story
)

# Display generation metrics
print("Story Generation Metrics:")
print(f"Time taken: {story_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Input tokens: {story_response.get('usage', {}).get('inputTokens', 0)}")
print(f"Output tokens: {story_response.get('usage', {}).get('outputTokens', 0)}\n")

# Display the story
display(Markdown("## Generated Story"))
display(Markdown(story_content))

### クリエイティブ ライティングの品質分析

ビジネス レポートとは異なり、クリエイティブ ライティングには異なる評価基準が必要です。分析してみましょう:
1. **物語構造**: ストーリーの流れと構築がどの程度優れているか
2. **キャラクター開発**: キャラクターがどの程度うまく描かれているか
3. **言語の使用**: 説明と会話の有効性
4. **テーマ要素**: テーマがどの程度うまく展開されているか

In [ ]:
def display_creative_analysis(metrics):
    """Display creative writing analysis in a readable format"""
    display(Markdown(f"""
    ### Creative Writing Analysis
    
    #### Structure Metrics
    - Total Words: {metrics['total_words']}
    - Paragraphs: {metrics['paragraphs']}
    - Scene Breaks: {metrics['scene_breaks']}
    
    #### Content Balance
    - Dialogue Lines: {metrics['dialogue_lines']}
    - Dialogue Percentage: {metrics['dialogue_percentage']:.1f}%
    - Descriptive Paragraphs: {metrics['descriptive_paragraphs']}
    
    #### Recommendations
    - Ideal dialogue percentage is typically 25-40% for engaging stories
    - Aim for a descriptive paragraph every 2-3 paragraphs
    - Consider adding more scene breaks if the story spans multiple scenes
    """))

In [ ]:
# Analyze our generated story
story_metrics = claude_utils.analyze_creative_writing(story_content)
display_creative_analysis(story_metrics)

# Let's create an enhanced creative prompt based on this analysis
def create_enhanced_creative_prompt(prompt, metrics):
    """
    Create an improved creative writing prompt based on analysis
    """
    current_dialogue_pct = metrics["dialogue_percentage"]
    dialogue_guidance = ("increase dialogue" if current_dialogue_pct < 25 
                        else "reduce dialogue" if current_dialogue_pct > 40 
                        else "maintain current dialogue level")
    
    enhanced_prompt = f"""
    Please create a story based on:
    
    {prompt}
    
    Writing Guidelines:
    1. Structure:
       - Aim for {metrics['total_words'] // 5}-{metrics['total_words'] // 3} words per scene
       - Include clear scene transitions
       - Balance action and reflection
    
    2. Character Development:
       - Show character traits through actions and dialogue
       - Include internal thoughts and reactions
       - Develop character arc through the story
    
    3. Dialogue and Description:
       - {dialogue_guidance} to achieve 25-40% dialogue
       - Include a descriptive paragraph every 2-3 paragraphs
       - Use all five senses in descriptions
    
    4. Pacing:
       - Start with a hook
       - Build tension gradually
       - Include 2-3 turning points
       - Provide satisfying resolution
    
    Format using markdown with scene breaks (***) between major sections.
    """
    
    return enhanced_prompt

In [ ]:
# Create enhanced prompt using our analysis
enhanced_story_prompt = create_enhanced_creative_prompt(story_prompt, story_metrics)

# Generate improved story with larger reasoning budget
improved_story_content, improved_story_response = generate_creative_content(
    enhanced_story_prompt,
    content_type="story",
    reasoning_budget=10240,  # Increased for more thorough development
    max_tokens=4000        # Increased for more detailed story
)

# Analyze the improved story
improved_story_metrics = claude_utils.analyze_creative_writing(improved_story_content)

# Compare original and improved stories
print("Comparison of Original vs. Improved Story:")
print(f"Original Story Words: {story_metrics['total_words']}")
print(f"Original Dialogue %: {story_metrics['dialogue_percentage']:.1f}%")
print(f"Improved Story Words: {improved_story_metrics['total_words']}")
print(f"Improved Dialogue %: {improved_story_metrics['dialogue_percentage']:.1f}%\n")

# Display the improved story
display(Markdown("## Improved Story"))
display(Markdown(improved_story_content))

### 高度な創作ライティング テクニック

基本的なストーリー生成を改善する方法を確認したので、高度なテクニックをいくつか見てみましょう。

1. **複数の視点**: さまざまなキャラクターの視点でストーリーを生成する
2. **複雑なプロット構造**: 非線形の物語を作成する
3. **世界構築**: 詳細な設定と環境を開発する
4. **キャラクター アーク**: 長い作品でのキャラクター開発を管理する

これらの高度な機能を創作ライティング ジェネレーターに実装してみましょう。

In [ ]:
def generate_advanced_story(
    prompt,
    viewpoints=None,      # List of character viewpoints
    plot_structure="linear",  # linear, nonlinear, parallel
    world_building_depth="medium",  # light, medium, detailed
    reasoning_budget=12288,
    max_tokens=5000
):
    """
    Generate a story with advanced creative writing features
    
    Args:
        prompt (str): Base story prompt
        viewpoints (list): Character viewpoints to include
        plot_structure (str): Type of plot structure
        world_building_depth (str): Level of setting detail
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        tuple: (content, response object)
    """
    # Create structured prompt for advanced story
    advanced_prompt = f"""
    Create an advanced story incorporating these elements:
    
    Base Prompt:
    {prompt}
    
    Technical Requirements:
    1. Plot Structure: {plot_structure}
    2. World Building Depth: {world_building_depth}
    """
    
    # Add viewpoint requirements if specified
    if viewpoints:
        advanced_prompt += f"\n3. Viewpoint Characters:\n"
        for char in viewpoints:
            advanced_prompt += f"   - {char}\n"
    
    advanced_prompt += """
    Development Guidelines:
    - Create distinct voice for each viewpoint
    - Maintain consistent world details
    - Weave plot threads together naturally
    - Balance exposition and action
    - Use scene breaks (***) between viewpoints
    """
    
    # Generate the story
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        advanced_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### 高度なストーリー生成のテスト

次のことを示すより複雑なストーリーを作成しましょう:
- 複数のキャラクターの視点
- 非線形のプロット構造
- 詳細な世界構築

これらの要素を効果的に探求できる SF の前提を使用します。

In [ ]:
# Define our advanced story parameters
advanced_story_prompt = """
Create a story about the first contact between humanity and an alien civilization, 
told from multiple perspectives as strange objects appear simultaneously in different parts of the world.

Setting elements:
- Near-future Earth (2045)
- Advanced AI monitoring systems
- Quantum communication technology
- Global climate challenges

Key events should be shown from different viewpoints to build a complete picture of what's happening.
"""

# Define viewpoint characters
viewpoints = [
    "Dr. Sarah Chen - Quantum physicist at CERN",
    "Captain Maya Rodriguez - Space Force orbital defense",
    "AI System GAIA-9 - Global monitoring network",
    "Diplomat Kwame Osei - UN First Contact Protocol leader"
]

# Generate advanced story
complex_story_content, complex_story_response = generate_advanced_story(
    prompt=advanced_story_prompt,
    viewpoints=viewpoints,
    plot_structure="nonlinear",
    world_building_depth="detailed",
    reasoning_budget=16384,  # Large budget for complex narrative
    max_tokens=6000         # Extended length for multiple viewpoints
)

# Analyze the complex story
complex_story_metrics = claude_utils.analyze_creative_writing(complex_story_content)

# Display metrics
print("Advanced Story Generation Metrics:")
print(f"Time taken: {complex_story_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Total words: {complex_story_metrics['total_words']}")
print(f"Scene breaks: {complex_story_metrics['scene_breaks']}")
print(f"Dialogue percentage: {complex_story_metrics['dialogue_percentage']:.1f}%\n")

# Display the story
display(Markdown("## Advanced Multi-Viewpoint Story"))
display(Markdown(complex_story_content))

### チェックイン ポイント: 高度なクリエイティブ ライティング

複雑なクリエイティブ コンテンツの作成について学んだことを復習しましょう。

**主要なテクニック:**
1. **視点管理**: キャラクターの個性を維持する方法
2. **プロットの複雑さ**: 非線形の物語を効果的に処理する
3. **世界構築**: 一貫性のある詳細な設定を作成する
4. **ストーリーのバランス**: 複数のスレッドにわたるペースを管理する

**検討すべき質問:**
- 推論予算の増加は物語の複雑さにどのように影響しましたか?
- どの視点の遷移が最も効果的でしたか?
- 世界構築の詳細は一貫性をどの程度維持しましたか?
- これらのテクニックからメリットを得られる他のストーリー タイプはどれですか?

次に、最後のコンテンツ タイプである技術ドキュメントに移りましょう。

## 3. 技術文書の作成

技術文書の作成には、レポートやクリエイティブな文章とは異なるアプローチが必要です。主な要素は次のとおりです。
- 明確な構造と構成
- 正確な技術用語
- コード例と説明
- ステップバイステップの手順

正確性と明瞭性を維持する包括的な技術文書を作成するためのフレームワークを作成しましょう。

In [ ]:
def generate_technical_doc(
    prompt,
    doc_type="guide",  # guide, api_doc, tutorial
    include_code=True,
    technical_level="intermediate",  # beginner, intermediate, advanced
    reasoning_budget=8192,
    max_tokens=4000
):
    """
    Generate technical documentation using Claude's extended thinking
    
    Args:
        prompt (str): The documentation requirements
        doc_type (str): Type of technical document
        include_code (bool): Whether to include code examples
        technical_level (str): Target expertise level
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        tuple: (content, response object)
    """
    # Create structured prompt for technical documentation
    tech_prompt = f"""
    Please create {doc_type} technical documentation for the following:
    
    {prompt}
    
    Target audience: {technical_level} level
    
    Documentation structure:
    1. Overview
    2. Prerequisites
    3. Technical Background
    4. Implementation Details
    5. Usage Examples
    6. Best Practices
    7. Troubleshooting
    
    Requirements:
    - Use clear, precise technical language
    - Include appropriate code examples
    - Provide step-by-step instructions where needed
    - Add notes and warnings for important considerations
    - Use markdown formatting for readability
    
    Format code examples with appropriate language tags for syntax highlighting.
    """
    
    # Invoke Claude with extended thinking
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        tech_prompt,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=max_tokens
    )
    
    return claude_utils.extract_response_content(response), response

### 技術ドキュメント生成のテスト

実際の例でドキュメント生成ツールをテストしてみましょう。REST API エンドポイントのドキュメントを作成します。これにより、Claude が次のことができることが示されます。
- 技術概念を明確に説明する
- 正確なコード例を提供する
- 情報を論理的に構造化する
- 一般的なユースケースと問題に対処する

In [ ]:
# Define our API documentation prompt
api_doc_prompt = """
Create documentation for a user authentication API endpoint with the following specifications:

Endpoint: /api/v1/auth/login
Method: POST
Purpose: Authenticate users and return JWT token

Technical Details:
- Rate limiting: 5 requests per minute
- Token expiration: 24 hours
- Required fields: email, password
- Optional fields: remember_me
- Supports both JSON and form-data

Include:
- Request/response formats
- Authentication flows
- Error handling
- Security considerations
- Code examples in Python and JavaScript
"""

# Generate the API documentation
api_doc_content, api_doc_response = generate_technical_doc(
    prompt=api_doc_prompt,
    doc_type="api_doc",
    technical_level="intermediate",
    reasoning_budget=8192,
    max_tokens=4000
)

# Display generation metrics
print("Technical Documentation Generation Metrics:")
print(f"Time taken: {api_doc_response.get('_elapsed_time', 0):.2f} seconds")
print(f"Input tokens: {api_doc_response.get('usage', {}).get('inputTokens', 0)}")
print(f"Output tokens: {api_doc_response.get('usage', {}).get('outputTokens', 0)}\n")

# Display the documentation
display(Markdown("## API Documentation"))
display(Markdown(api_doc_content))

In [ ]:
def display_technical_analysis(metrics):
    """Display technical documentation analysis"""
    display(Markdown(f"""
    ### Technical Documentation Analysis
    
    #### Content Structure
    - Total Words: {metrics['total_words']}
    - Main Sections: {metrics['sections']}
    - Subsections: {metrics['subsections']}
    
    #### Technical Elements
    - Code Examples: {metrics['code_blocks']}
    - Usage Examples: {metrics['examples']}
    - Warnings/Notes: {metrics['warnings']}
    - References/Links: {metrics['links']}
    
    #### Recommendations
    - Aim for 2-3 code examples per major concept
    - Include warning notes for common pitfalls
    - Add cross-references where appropriate
    """))

In [ ]:
# Analyze our generated API documentation
tech_doc_metrics = claude_utils.analyze_technical_doc(api_doc_content)
display_technical_analysis(tech_doc_metrics)

# Create an enhanced prompt based on this analysis
def create_enhanced_tech_doc_prompt(prompt, metrics):
    """
    Create an improved technical documentation prompt based on analysis
    """
    enhanced_prompt = f"""
    Please create detailed technical documentation for:
    
    {prompt}
    
    Documentation Requirements:
    1. Structure:
       - Include {max(metrics['sections'] + 2, 7)} main sections
       - Create {max(metrics['subsections'] + 3, 10)} subsections for detailed topics
       - Add a table of contents at the start
    
    2. Technical Content:
       - Provide at least {max(metrics['code_blocks'] + 2, 5)} code examples
       - Include {max(metrics['examples'] + 2, 4)} usage examples
       - Add {max(metrics['warnings'] + 2, 5)} warning/note boxes for important considerations
    
    3. Clarity and Depth:
       - Start each section with a brief overview
       - Include practical examples after each concept
       - Provide troubleshooting guidance
       - Add security best practices
    
    4. Formatting:
       - Use markdown headers for structure (# for main, ## for sub)
       - Format code blocks with language-specific syntax highlighting
       - Use tables for parameter listings
       - Add blockquotes for important notes
    
    The documentation should be thorough yet practical, suitable for developers implementing the API.
    """
    
    return enhanced_prompt

In [ ]:
# Generate improved API documentation
enhanced_api_prompt = create_enhanced_tech_doc_prompt(api_doc_prompt, tech_doc_metrics)

improved_api_doc_content, improved_api_doc_response = generate_technical_doc(
    prompt=enhanced_api_prompt,
    doc_type="api_doc",
    technical_level="intermediate",
    reasoning_budget=12288,  # Increased for more thorough documentation
    max_tokens=5000         # Increased for more detailed content
)

# Analyze the improved documentation
improved_tech_doc_metrics = claude_utils.analyze_technical_doc(improved_api_doc_content)

# Compare original and improved documentation
print("Comparison of Original vs. Improved Documentation:")
print(f"Original Words: {tech_doc_metrics['total_words']}")
print(f"Original Code Examples: {tech_doc_metrics['code_blocks']}")
print(f"Improved Words: {improved_tech_doc_metrics['total_words']}")
print(f"Improved Code Examples: {improved_tech_doc_metrics['code_blocks']}\n")

# Display the improved documentation
display(Markdown("## Improved API Documentation"))
display(Markdown(improved_api_doc_content))

## チェックイン ポイント: 技術ドキュメント

効果的な技術ドキュメントの生成について学んだことを確認しましょう:

**重要な要素:**
1. **構造**: 適切なセクションとサブセクションによる明確な構成
2. **コード例**: 実用的で、十分にコメントされた実装例
3. **使用ガイド**: 明確な指示とベスト プラクティス
4. **エラー処理**: 包括的なトラブルシューティング情報

**検討すべき質問:**
- 強化された構造によってドキュメントの明瞭性はどのように向上しましたか?
- ドキュメントのどの側面が開発者にとって最も役立ちますか?
- このアプローチからメリットを得られる他の種類の技術ドキュメントは?
- コード例の生成をさらに改善するにはどうすればよいですか?

では、長い形式の出力の管理と検証に関する最後のセクションに移りましょう。

## 4. 長文出力の管理と検証

長文コンテンツを生成する場合、次の点が重要です:
- ドキュメント全体の一貫性を確保する
- 技術的な正確性を検証する
- 完全性を確認する
- 構造的整合性を維持する

長文コンテンツの生成を管理および検証するためのツールを作成しましょう。

In [ ]:
def display_validation_results(validation):
    """Display content validation results in a readable format"""
    status = "✅ PASSED" if validation["pass"] else "❌ FAILED"
    
    display(Markdown(f"""
    ### Content Validation Results: {status}
    
    #### Document Structure
    - Total Length: {validation['total_length']} characters
    - Number of Sections: {validation['section_count']}
    
    #### Section Analysis
    """))
    
    for section, data in validation["sections"].items():
        status = "✅" if data["meets_min_length"] else "❌"
        display(Markdown(f"""
        **{section}** {status}
        - Length: {data['length']} characters
        - Paragraphs: {data['paragraphs']}
        """))
    
    if validation["issues"]:
        display(Markdown("\n#### Issues Found ❌"))
        for issue in validation["issues"]:
            display(Markdown(f"- {issue}"))
    
    if validation["warnings"]:
        display(Markdown("\n#### Warnings ⚠️"))
        for warning in validation["warnings"]:
            display(Markdown(f"- {warning}"))

In [ ]:
# Let's validate our different types of content

# 1. Validate the business report
report_validation = claude_utils.validate_long_form_content(
    report_content,
    content_type="report",
    required_sections=[
        "Executive Summary",
        "Background and Context",
        "Analysis and Findings",
        "Recommendations",
        "Implementation Plan",
        "Conclusion"
    ],
    min_section_length=200
)

print("Business Report Validation:")
display_validation_results(report_validation)

# 2. Validate the creative story
story_validation = claude_utils.validate_long_form_content(
    complex_story_content,
    content_type="creative",
    min_section_length=300  # Longer sections for creative writing
)

print("\nCreative Story Validation:")
display_validation_results(story_validation)

# 3. Validate the technical documentation
tech_doc_validation = claude_utils.validate_long_form_content(
    improved_api_doc_content,
    content_type="technical",
    required_sections=[
        "Overview",
        "Prerequisites",
        "Technical Background",
        "Implementation Details",
        "Usage Examples",
        "Best Practices",
        "Troubleshooting"
    ],
    min_section_length=150
)

print("\nTechnical Documentation Validation:")
display_validation_results(tech_doc_validation)

### 検証に基づいてコンテンツを改善する

コンテンツの潜在的な問題を特定したので、次の方法でそれらの問題に対処するための機能を作成しましょう。
1. 短いセクションを拡張する
2. 不足している必須コンテンツを追加する
3. コンテンツ タイプのバランスをとる (ダイアログ、コード例など)
4. セクション間で一貫性を維持する

In [ ]:
def improve_content_based_on_validation(content, validation_results, content_type="general"):
    """
    Create a prompt to improve content based on validation results
    
    Args:
        content (str): Original content
        validation_results (dict): Results from content validation
        content_type (str): Type of content being improved
        
    Returns:
        str: Enhanced prompt for content improvement
    """
    improvement_prompt = f"""
    Please improve the following content, addressing these specific issues:
    
    Original Content:
    {content}
    
    Required Improvements:
    """
    
    # Add specific improvements based on validation issues
    if validation_results["issues"]:
        improvement_prompt += "\nCritical Issues to Fix:\n"
        for issue in validation_results["issues"]:
            improvement_prompt += f"- {issue}\n"
    
    if validation_results["warnings"]:
        improvement_prompt += "\nWarnings to Address:\n"
        for warning in validation_results["warnings"]:
            improvement_prompt += f"- {warning}\n"
    
    # Add content-type specific guidance
    if content_type == "technical":
        improvement_prompt += """
        Technical Documentation Requirements:
        - Ensure each section is comprehensive and clearly explained
        - Add or expand code examples where missing
        - Include error handling and edge cases
        - Add more practical usage examples
        """
    elif content_type == "creative":
        improvement_prompt += """
        Creative Writing Requirements:
        - Balance dialogue and narrative
        - Enhance descriptive passages
        - Strengthen character development
        - Improve scene transitions
        """
    elif content_type == "report":
        improvement_prompt += """
        Report Requirements:
        - Strengthen data analysis
        - Make recommendations more specific
        - Add implementation details
        - Enhance executive summary
        """
    
    improvement_prompt += """
    
    Please maintain the original structure while expanding and improving the content.
    Ensure consistency across all sections and maintain the document's tone and style.
    """
    
    return improvement_prompt

In [ ]:
# Create improvement prompt for technical documentation
tech_improvement_prompt = improve_content_based_on_validation(
    improved_api_doc_content,
    tech_doc_validation,
    content_type="technical"
)

# Generate improved version with higher reasoning budget
final_tech_doc_content, final_tech_response = generate_technical_doc(
    prompt=tech_improvement_prompt,
    doc_type="api_doc",
    technical_level="intermediate",
    reasoning_budget=16384,  # Increased for more thorough improvements
    max_tokens=6000         # Increased to allow for expanded content
)

# Validate the final version
final_tech_validation = claude_utils.validate_long_form_content(
    final_tech_doc_content,
    content_type="technical",
    required_sections=[
        "Overview",
        "Prerequisites",
        "Technical Background",
        "Implementation Details",
        "Usage Examples",
        "Best Practices",
        "Troubleshooting"
    ],
    min_section_length=150
)

# Compare versions
print("Documentation Version Comparison:")
print(f"Original Length: {len(improved_api_doc_content)} characters")
print(f"Final Length: {len(final_tech_doc_content)} characters")
print(f"Original Issues: {len(tech_doc_validation['issues'])}")
print(f"Final Issues: {len(final_tech_validation['issues'])}\n")

# Display final validation results
display(Markdown("### Final Documentation Validation"))
display_validation_results(final_tech_validation)

# Display the final improved documentation
display(Markdown("## Final Improved API Documentation"))
display(Markdown(final_tech_doc_content))

## 長文コンテンツ生成の管理: ベスト プラクティス

さまざまなコンテンツ タイプと検証アプローチの実験に基づいて、長文コンテンツの生成と管理に関する主なベスト プラクティスを以下に示します。

1. **生成前の計画**
- 明確なセクション要件を定義する
- 適切な長さのガイドラインを設定する
- コンテンツ タイプ固有の要素を指定する
- 合理的な推論予算を構成する

2. **コンテンツ生成**
- 構造化されたプロンプトを使用する
- 十分な推論予算を有効にする
- 検証要件を含める
- 特定の例/要素を要求する

3. **検証と改善**
- 構造の完全性を確認する
- コンテンツ固有の要素を検証する
- 問題に体系的に対処する
- 改善の一貫性を維持する

4. **品質管理**
- 定期的な検証チェック
- 段階的な改善
- コンテンツ タイプ固有のメトリック
- 一貫性の検証

## 最終チェックイン ポイント: 長文コンテンツの生成

このレッスンで取り上げた主要な概念を確認しましょう:

**1. コンテンツ タイプ戦略**
- ビジネス レポート: 構造、分析、推奨事項
- クリエイティブ ライティング: キャラクター開発、プロット、物語の流れ
- 技術ドキュメント: 明瞭性、正確性、実用的な例

**2. 拡張思考の適用**
- 複雑なコンテンツに大規模な推論予算を使用する
- 包括的な出力のためのプロンプトの構造化
- 長いコンテンツ全体でコンテンツの一貫性を管理する

**3. 検証と改善**
- 自動コンテンツ検証
- 問題の特定と解決
- コンテンツ固有の品質メトリック
- 体系的な改善プロセス

**検討すべき質問:**
- 拡張思考から最も恩恵を受けたコンテンツ タイプはどれですか?
- 検証はコンテンツの品質向上にどのように役立ちましたか?
- これらの手法を使用できる他のタイプの長文コンテンツはどれですか?
- これらのアプローチを特定のニーズに合わせてどのように適応させますか?

## 結論と次のステップ

### 重要なポイント
1. **拡張思考力**: Claude 3.7 の拡張思考機能により、高品質で一貫性のある長文コンテンツの生成が可能になります
2. **コンテンツ タイプの最適化**: コンテンツ タイプによって、異なるアプローチと検証戦略が必要になります
3. **品質管理**: 長文コンテンツには、体系的な検証と改善のプロセスが不可欠です
4. **反復的な改善**: 段階的な改良により、出力の品質が向上します

### 実用的なアプリケーション
- ビジネス レポートの生成
- クリエイティブ ライティングとストーリーテリング
- 技術ドキュメント
- 教育コンテンツ
- 研究論文
- 製品の説明
- ポリシー ドキュメント

### 次のステップ
1. さまざまなコンテンツ タイプで**実験**します
2. ニーズに合わせて**検証基準を**カスタマイズ**します
3. ユース ケースに特化した**プロンプトを開発**します
4. コンテンツ生成用の**自動ワークフローを構築**します

### 追加リソース
- Claude 3.7ドキュメント
- コンテンツ生成のベストプラクティス
- 拡張思考最適化ガイド
- 検証フレームワークのドキュメント

## 練習問題

学習した内容を強化するために、次の練習問題に取り組んでください。

1. **ビジネス レポートの作成**

- 市場分析レポートを作成する
- プロジェクト提案を作成する
- 戦略計画を作成する

2. **クリエイティブ ライティング**

- 複数の視点から短編小説を書く
- 詳細なシーンの説明を作成する
- キャラクターの背景を作成する

3. **技術ドキュメント**
- 複雑な API をドキュメント化する
- ユーザー ガイドを作成する
- インストール手順を書く

4. **コンテンツの検証**
- カスタム検証基準を開発する
- 専用の改善プロンプトを作成する
- 自動検証ワークフローを構築する

これらの練習問題に取り組むときは、このレッスンで説明した手法、検証ツール、ベスト プラクティスを忘れずに使用してください。